In [33]:
import cv2
import numpy as np
from motpy import Detection, MultiObjectTracker

# Haar Cascadeファイルのロード
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
lefteye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lefteye_2splits.xml')
righteye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_righteye_2splits.xml')

# カメラの指定
cap = cv2.VideoCapture(0)

# トラッカーの作成
tracker = MultiObjectTracker(dt=0.1)

# 各パーツのリサイズ比を入力
eyex, eyey = 1.1, 1.3

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # グレースケール画像に変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 顔を検出
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # 検出されたバウンディングボックスをmotpyに渡す
    detections = []
    for (x, y, w, h) in faces:
        xmin, ymin, xmax, ymax = x, y, x + w, y + h
        detections.append(Detection(box=[xmin, ymin, xmax, ymax]))

        # 顔領域の取得
        face = frame[y:y + h, x:x + w]
        face_gray = gray[y:y + h, x:x + w]

        # 左目の検出と拡大
        lefteye = lefteye_cascade.detectMultiScale(face_gray)
        for (ex, ey, ew, eh) in lefteye:
            eye_roi = face[ey:ey + eh, ex:ex + ew]
            big_eye = cv2.resize(eye_roi, None, fx=eyex, fy=eyey)
            
            # 新しいサイズが元の領域を越えないように調整
            new_height, new_width = big_eye.shape[:2]
            target_height = ey + new_height if ey + new_height < face.shape[0] else face.shape[0]
            target_width = ex + new_width if ex + new_width < face.shape[1] else face.shape[1]
            face[ey:target_height, ex:target_width] = big_eye[:(target_height - ey), :(target_width - ex)]

        # 右目の検出と拡大
        righteye = righteye_cascade.detectMultiScale(face_gray)
        for (ex, ey, ew, eh) in righteye:
            eye_roi = face[ey:ey + eh, ex:ex + ew]
            big_eye = cv2.resize(eye_roi, None, fx=eyex, fy=eyey)
            
            # 新しいサイズが元の領域を越えないように調整
            new_height, new_width = big_eye.shape[:2]
            target_height = ey + new_height if ey + new_height < face.shape[0] else face.shape[0]
            target_width = ex + new_width if ex + new_width < face.shape[1] else face.shape[1]
            face[ey:target_height, ex:target_width] = big_eye[:(target_height - ey), :(target_width - ex)]

    # トラッカーに検出結果を渡して追跡
    tracker.step(detections=detections)

    # 追跡中の顔をフレーム上に描画
    for track in tracker.active_tracks():
        box = track.box  # トラッキングボックス
        xmin, ymin, xmax, ymax = int(box[0]), int(box[1]), int(box[2]), int(box[3])
        track_id = track.id

        # バウンディングボックスとIDを描画
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # フレームを表示
    cv2.imshow('Real-time Face Modification', frame)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 終了処理
cap.release()
cv2.destroyAllWindows()
